In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random as rn
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
path =   # Enter the directory of your project folder
PATH = path + "/Results"

if not os.path.exists(PATH):
    os.mkdir(PATH) 

In [ ]:
df = pd.read_csv(path+"//ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
df1 = pd.read_csv(PATH+"//Data one hot encoded.csv").drop(['Unnamed: 0'],axis=1)
clientID = df.ClientID.unique()

df["EntryDate"] = pd.to_datetime(df['EntryDate'])
df["ExitDate"] = pd.to_datetime(df['ExitDate'])
df = df.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)
df1["EntryDate"] = pd.to_datetime(df1['EntryDate'])
df1["ExitDate"] = pd.to_datetime(df1['ExitDate'])
df1 = df1.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)

In [ ]:
# separating the living situation and the exit destination into categories
goal = [10,11]
closer = [3,19,20,21,22,23,25,26,28,31]
trans = [1,2,4,5,6,12,13,14,15,18,27,29]
no_progress = [7,16]
hard_to_judge = [8,9,17,24,30,99]

In [ ]:
#separating out data for determining transition matrix

data = pd.DataFrame(columns=['ClientID',"start",'end'])
for i in range(len(clientID)):
    temp = df[df['ClientID']==clientID[i]].reset_index(drop=True)
    for j in range(len(temp)):
        if j == 0:
            start = 'entry'
            end = temp['ProjectType'][j]
        elif j == len(temp)-1:
            start = temp['ProjectType'][j]
            end = 'exit'
        else:
            start = temp['ProjectType'][j]
            end = temp['ProjectType'][j+1]
        data = data.append({'ClientID':clientID[i],'start':start,'end':end},ignore_index=True)
    

In [ ]:
# calculatin probability
edge_prob = pd.DataFrame(columns=["edge",'start','end','numberOfEdges'])
edges = pd.Series([tuple([data['start'].iloc[i],data['end'].iloc[i]]) for i in range(len(data))])
edges_unique = edges.unique()

for i in edges_unique:
    edgeNo = len(edges[edges==i])
    edge_prob = edge_prob.append({'edge':i,'start':i[0],"end":i[1],'numberOfEdges':edgeNo},ignore_index=True)

total_prob = edge_prob.groupby("start").sum()['numberOfEdges']
edge_prob['total'] = np.nan

for i in range(len(edge_prob)):
    edge_prob['total'].loc[i] = total_prob.loc[edge_prob['start'].loc[i]]
    
edge_prob['prob'] = edge_prob['numberOfEdges']/edge_prob['total']

In [ ]:
transition_matrix = pd.DataFrame(index = edge_prob.start.unique(),columns = edge_prob.end.unique())

for i in transition_matrix.index:
    for j in transition_matrix.columns:
        if (i,j) not in list(edge_prob['edge']):
            continue
        transition_matrix.loc[i][j] = edge_prob[edge_prob['edge']==(i,j)]['prob'].values[0]

In [ ]:
# effective trajectory, exit and entry
data = pd.DataFrame(columns=['ClientID','Entry',"Effective path","Exit",'Path length'])
for i in range(len(clientID)):
    temp = df[df['ClientID']==clientID[i]].reset_index(drop=True)
    eff_path = temp.ProjectType.unique()
    data = data.append({'ClientID':clientID[i],'Entry':temp['LivingSituation'].iloc[0],'Effective path':eff_path,'Exit':temp['Destination'].iloc[-1],'Path length':len(eff_path)},ignore_index=True)
    

In [ ]:
degree_info = pd.DataFrame(columns=['projectType','outdegree','indegree'])
projectType = transition_matrix.columns.values 
projectType = np.insert(projectType,0,'entry')

for i in range(len(projectType)):
    if projectType[i] not in transition_matrix.columns: 
        indegree = 0
    else:
        indegree = len(transition_matrix[projectType[i]][transition_matrix[projectType[i]].notna()])
    if projectType[i] not in transition_matrix.index:
        outdegree = 0
    else:
        outdegree = len(transition_matrix.loc[projectType[i]][transition_matrix.loc[projectType[i]].notna()])
    degree_info = degree_info.append({'projectType':projectType[i],'outdegree':outdegree,'indegree':indegree},ignore_index=True)
   

In [ ]:
degree_info

In [ ]:
degree = pd.DataFrame(columns=['ClientID','first_node','last_node','outdegree_first',"indegree_last",'exit_type','length'])
for i in range(len(data)):
    temp = data.iloc[i]
    first = temp['Effective path'][0]
    last = temp['Effective path'][-1]
    if temp['Exit'] in goal:
        degree = degree.append({'ClientID':temp.ClientID,'first_node':first,"last_node":last,'outdegree_first':degree_info[degree_info['projectType']==first]["outdegree"].values[0],'indegree_last':degree_info[degree_info['projectType']==last]["indegree"].values[0],'exit_type':'successful','length':temp['Path length']},ignore_index=True)
    elif np.isnan(temp['Exit']):
        degree = degree.append({'ClientID':temp.ClientID,'first_node':first,"last_node":last,'outdegree_first':degree_info[degree_info['projectType']==first]["outdegree"].values[0],'indegree_last':degree_info[degree_info['projectType']==last]["indegree"].values[0],'exit_type':'abandoned','length':temp['Path length']},ignore_index=True)
    else:
        degree = degree.append({'ClientID':temp.ClientID,'first_node':first,"last_node":last,'outdegree_first':degree_info[degree_info['projectType']==first]["outdegree"].values[0],'indegree_last':degree_info[degree_info['projectType']==last]["indegree"].values[0],'exit_type':'unsuccessful','length':temp['Path length']},ignore_index=True)
    
   

In [ ]:
degree_avg = pd.DataFrame(columns=["exit_type",'length','outdegree','indegree'])
length_unique = degree.length.unique()
for i in range(len(length_unique)):
    temp = degree[degree['length']==length_unique[i]]
    temp1 = temp[temp['exit_type']=='successful']
    temp2 = temp[temp['exit_type']=='unsuccessful']    
    temp3 = temp[temp['exit_type']=='abandoned']    
    outdegreeMean_s = temp1.outdegree_first.mean()
    indegreeMean_s = temp1.indegree_last.mean()
    outdegreeMean_u = temp2.outdegree_first.mean()
    indegreeMean_u = temp2.indegree_last.mean()
    outdegreeMean_a = temp3.outdegree_first.mean()
    indegreeMean_a = temp3.indegree_last.mean()
    
    degree_avg = degree_avg.append({"exit_type":'successful','length':length_unique[i],'outdegree':outdegreeMean_s,'indegree':indegreeMean_s},ignore_index=True)
    degree_avg = degree_avg.append({"exit_type":'unsuccessful','length':length_unique[i],'outdegree':outdegreeMean_u,'indegree':indegreeMean_u},ignore_index=True)
    degree_avg = degree_avg.append({"exit_type":'abandoned','length':length_unique[i],'outdegree':outdegreeMean_a,'indegree':indegreeMean_a},ignore_index=True)


    
    
    

In [ ]:
degree_avg.groupby('exit_type').mean()